In [126]:
import json
import requests
from datetime import datetime
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [127]:
response = requests.get('https://date.nager.at/api/v3/publicholidays/2024/BR')
public_holidays = json.loads(response.content)

for public_holiday in public_holidays:
  print(public_holiday['date'])

2024-01-01
2024-02-12
2024-02-13
2024-03-29
2024-03-31
2024-04-21
2024-05-01
2024-05-30
2024-07-09
2024-09-07
2024-10-12
2024-11-02
2024-11-15
2024-12-25


**1. Quantos feriados há no Brasil em todo o ano de 2024?**

In [128]:
quantidade_feriados = 0

for public_holiday in public_holidays:
  quantidade_feriados=quantidade_feriados+1

print(quantidade_feriados)

14


**2. Qual mês de 2024 tem o maior número de feriados?**

In [129]:
feriados_por_mes = {i: 0 for i in range(1, 13)}
for public_holiday in public_holidays:
    mes = int(public_holiday['date'].split('-')[1])
    feriados_por_mes[mes] += 1

In [130]:
for mes, quantidade in feriados_por_mes.items():
    print(f"Mês {mes}: {quantidade} feriados")

Mês 1: 1 feriados
Mês 2: 2 feriados
Mês 3: 2 feriados
Mês 4: 1 feriados
Mês 5: 2 feriados
Mês 6: 0 feriados
Mês 7: 1 feriados
Mês 8: 0 feriados
Mês 9: 1 feriados
Mês 10: 1 feriados
Mês 11: 2 feriados
Mês 12: 1 feriados


**3. Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?**

In [131]:
feriados_semana = 0

for public_holiday in public_holidays:
    data = datetime.strptime(public_holiday['date'], '%Y-%m-%d')
    if data.weekday() < 5:
        feriados_semana += 1

print(feriados_semana)

9


**4. Qual foi a temperatura média em cada mês?**

In [132]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": -22.908333,
	"longitude": -43.196388,
	"start_date": "2024-01-01",
	"end_date": "2024-08-01",
	"hourly": "temperature_2m"
}

responses = openmeteo.weather_api(url, params=params)

response = responses[0]

latitude = response.Latitude()
longitude = response.Longitude()
elevation = response.Elevation()
timezone = response.Timezone()
timezone_abbreviation = response.TimezoneAbbreviation()
utc_offset_seconds = response.UtcOffsetSeconds()

hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature_2m
}

hourly_dataframe = pd.DataFrame(data=hourly_data)

print(hourly_dataframe)

                          date  temperature_2m
0    2024-01-01 00:00:00+00:00       24.441999
1    2024-01-01 01:00:00+00:00       23.392000
2    2024-01-01 02:00:00+00:00       23.392000
3    2024-01-01 03:00:00+00:00       23.191999
4    2024-01-01 04:00:00+00:00       22.841999
...                        ...             ...
5131 2024-08-01 19:00:00+00:00       25.292000
5132 2024-08-01 20:00:00+00:00       23.642000
5133 2024-08-01 21:00:00+00:00       21.792000
5134 2024-08-01 22:00:00+00:00       20.642000
5135 2024-08-01 23:00:00+00:00       20.142000

[5136 rows x 2 columns]


- As coordenadas do Rio de Janeiro eu encontrei nesse site: https://www.latlong.net/place/rio-de-janeiro-brazil-27580.html#:~:text=Latitude%20and%20longitude%20coordinates%20are%3A%20%2D22.908333%2C%20%2D43.196388.
- O código acima eu encontrei na documentação da API.



In [133]:
temperatura_media_mensal = hourly_dataframe.resample('M', on='date').mean()
print(temperatura_media_mensal)

                           temperature_2m
date                                     
2024-01-31 00:00:00+00:00       26.741865
2024-02-29 00:00:00+00:00       27.258522
2024-03-31 00:00:00+00:00       26.484541
2024-04-30 00:00:00+00:00       25.206236
2024-05-31 00:00:00+00:00       25.072308
2024-06-30 00:00:00+00:00       22.701305
2024-07-31 00:00:00+00:00       21.153963
2024-08-31 00:00:00+00:00       21.225332


**5. Qual foi o tempo predominante em cada mês nesse período?**

In [134]:
with open('descriptions/descriptions.json', 'r') as f:
    weather_code_description = json.load(f)
    
print(list(weather_code_description.keys())[:10]) 

['0', '1', '2', '3', '45', '48', '51', '53', '55', '56']


In [135]:
params = {
    "latitude": -22.908333,
    "longitude": -43.196388,
    "start_date": "2024-01-01",
    "end_date": "2024-08-01",
    "hourly": "temperature_2m,weathercode"
}

responses = openmeteo.weather_api(url, params=params)
response = responses[0]

weather_codes = response.Hourly().Variables(1).ValuesAsNumpy() 
temperatures = response.Hourly().Variables(0).ValuesAsNumpy() 
dates = pd.date_range(
    start=pd.to_datetime(response.Hourly().Time(), unit="s", utc=True),
    end=pd.to_datetime(response.Hourly().TimeEnd(), unit="s", utc=True),
    freq=pd.Timedelta(seconds=response.Hourly().Interval()),
    inclusive="left"
)

df = pd.DataFrame({
    "datetime": dates,
    "weather_code": weather_codes,
    "temperature_2m": temperatures
})

df["datetime"] = pd.to_datetime(df["datetime"])
df["month"] = df["datetime"].dt.month
df["year"] = df["datetime"].dt.year

with open('descriptions/descriptions.json', 'r') as f:
    weather_code_description = json.load(f)

def get_weather_description(code, hour):
    code_str = str(int(code))
    if code_str in weather_code_description:
        if 6 <= hour < 18:
            return weather_code_description[code_str]["day"]["description"]
        else:
            return weather_code_description[code_str]["night"]["description"]
    else:
        print(f"{code_str} não encontrado")
    return "Unknown"

df["weather_description"] = df.apply(lambda row: get_weather_description(row["weather_code"], row["datetime"].hour), axis=1)

print(df.head())

                   datetime  weather_code  temperature_2m  month  year  \
0 2024-01-01 00:00:00+00:00           3.0       24.441999      1  2024   
1 2024-01-01 01:00:00+00:00           2.0       23.392000      1  2024   
2 2024-01-01 02:00:00+00:00           2.0       23.392000      1  2024   
3 2024-01-01 03:00:00+00:00           2.0       23.191999      1  2024   
4 2024-01-01 04:00:00+00:00           3.0       22.841999      1  2024   

  weather_description  
0              Cloudy  
1       Partly Cloudy  
2       Partly Cloudy  
3       Partly Cloudy  
4              Cloudy  


In [136]:
monthly_predominant_weather = df.groupby(["year", "month"])["weather_code"].agg(lambda x: x.mode().iloc[0])
monthly_predominant_weather_description = monthly_predominant_weather.map(lambda x: get_weather_description(x, 12))  # Assume o meio do dia para o mês

print(monthly_predominant_weather_description)

year  month
2024  1        Mainly Sunny
      2        Mainly Sunny
      3               Sunny
      4               Sunny
      5               Sunny
      6               Sunny
      7               Sunny
      8               Sunny
Name: weather_code, dtype: object


**6. Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?**

In [137]:
datas_feriados = [pd.to_datetime(feriado['date']) for feriado in public_holidays]

df_feriados = df[df["datetime"].dt.date.isin([d.date() for d in datas_feriados])]

resultados = []
for data_feriado in datas_feriados:
    data_feriado_str = data_feriado.date()
    dados_feriado = df_feriados[df_feriados["datetime"].dt.date == data_feriado_str]
    
    if not dados_feriado.empty:
        temperatura_media = dados_feriado["temperature_2m"].mean()
        tempo_predominante = dados_feriado["weather_description"].mode().iloc[0]
    else:
        temperatura_media = None
        tempo_predominante = "Sem dados"

    resultados.append({
        "data": data_feriado_str,
        "temperatura_media": temperatura_media,
        "descricao_tempo": tempo_predominante
    })

resultados_df = pd.DataFrame(resultados)

print(resultados_df)


          data  temperatura_media descricao_tempo
0   2024-01-01          24.919083          Cloudy
1   2024-02-12          30.246164           Sunny
2   2024-02-13          30.616999    Mainly Clear
3   2024-03-29          24.908667          Cloudy
4   2024-03-31          24.792000           Clear
5   2024-04-21          23.102417           Clear
6   2024-05-01          28.027418           Clear
7   2024-05-30          20.925333          Cloudy
8   2024-07-09          21.641998          Cloudy
9   2024-09-07                NaN       Sem dados
10  2024-10-12                NaN       Sem dados
11  2024-11-02                NaN       Sem dados
12  2024-11-15                NaN       Sem dados
13  2024-12-25                NaN       Sem dados


7. Considere as seguintes suposições:

- O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
- Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
- O cidadão carioca só vai à praia quando não está com frio;
- O cidadão carioca também só vai à praia em dias com sol, evitando dias totalmente nublados ou chuvosos (considere weather_code para determinar as condições climáticas).

**Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?**

In [138]:
def classificar_feriado(row):
    temperatura_abaixo = row["temperatura_media"] < 20 if row["temperatura_media"] is not None else False
    descricao_aproveitavel = any(word in row["descricao_tempo"] for word in ["Sunny", "Clear"])

    if temperatura_abaixo or not descricao_aproveitavel:
        return "Não Aproveitável"
    else:
        return "Aproveitável"

resultados_df["aproveitavel_ou_nao"] = resultados_df.apply(classificar_feriado, axis=1)

In [139]:
resultados_df

,data,temperatura_media,descricao_tempo,aproveitavel_ou_nao
0,2024-01-01,24.919083,Cloudy,Não Aproveitável
1,2024-02-12,30.246164,Sunny,Aproveitável
2,2024-02-13,30.616999,Mainly Clear,Aproveitável
3,2024-03-29,24.908667,Cloudy,Não Aproveitável
4,2024-03-31,24.792000,Clear,Aproveitável
5,2024-04-21,23.102417,Clear,Aproveitável
6,2024-05-01,28.027418,Clear,Aproveitável
7,2024-05-30,20.925333,Cloudy,Não Aproveitável
8,2024-07-09,21.641998,Cloudy,Não Aproveitável
9,2024-09-07,NaN,Sem dados,Não Aproveitável


Os feriados do dia 1/1, 29/3, 30/5 e 9/7 não foram aproveitáveis, os outros sim.

**8. Qual foi o feriado "mais aproveitável" de 2024?**

O feriado "mais aproveitável" foi o do dia 12/2, pois, além de estar ensolarado, a temperatura média foi de 30 graus.